In [3]:
from pyspark.sql import SparkSession

# Step 1: Start SparkSession
spark = SparkSession.builder \
    .appName("SalesDataSQL") \
    .getOrCreate()

# Step 2: Load CSV into DataFrame
df = spark.read.option("header", "true").option("inferSchema", "true").csv(r"C:\Users\User\Desktop\sales_data.csv")

# Step 3: Register DataFrame as SQL temporary view
df.createOrReplaceTempView("sales")

# Step 4: Example SQL Queries

# Show all records
spark.sql("SELECT * FROM sales").show()

# Total sales by region
spark.sql("""
    SELECT region, SUM(total_price) as total_sales
    FROM sales
    GROUP BY region
    ORDER BY total_sales DESC
""").show()

# Average unit price per category
spark.sql("""
    SELECT category, AVG(unit_price) as avg_price
    FROM sales
    GROUP BY category
""").show()

# Total quantity sold by product
spark.sql("""
    SELECT product, SUM(quantity) as total_quantity
    FROM sales
    GROUP BY product
""").show()

# 1. Filter: Orders above $1000
spark.sql("""
    SELECT *
    FROM sales
    WHERE total_price > 1000
""").show()

# 2. Grouping & Sorting: Top customers by total spent
spark.sql("""
    SELECT customer_name, SUM(total_price) AS total_spent
    FROM sales
    GROUP BY customer_name
    ORDER BY total_spent DESC
""").show()

# 3. Derived Column: Revenue per unit (total_price / quantity)
spark.sql("""
    SELECT order_id, product, total_price, quantity, 
           ROUND(total_price / quantity, 2) AS revenue_per_unit
    FROM sales
""").show()

# 4. Date Parsing: Month-wise sales trend
spark.sql("""
    SELECT SUBSTRING(order_date, 1, 7) AS order_month,
           SUM(total_price) AS monthly_sales
    FROM sales
    GROUP BY order_month
    ORDER BY order_month
""").show()

# 5. Conditional Logic: Categorize order value
spark.sql("""
    SELECT order_id, customer_name, total_price,
           CASE
               WHEN total_price >= 1000 THEN 'High'
               WHEN total_price >= 500 THEN 'Medium'
               ELSE 'Low'
           END AS order_value_category
    FROM sales
""").show()

# 6. Window Function: Rank products by total sales
spark.sql("""
    SELECT product, SUM(total_price) AS product_sales,
           RANK() OVER (ORDER BY SUM(total_price) DESC) AS sales_rank
    FROM sales
    GROUP BY product
""").show()
# Step 5: Stop Spark session
spark.stop()


+--------+----------+-----------+-------------+------+----------+-----------+--------+----------+-----------+
|order_id|order_date|customer_id|customer_name|region|   product|   category|quantity|unit_price|total_price|
+--------+----------+-----------+-------------+------+----------+-----------+--------+----------+-----------+
|    1001|2023-01-15|       C001|     John Doe| North|    Laptop|Electronics|       2|       800|       1600|
|    1002|2023-01-17|       C002|   Jane Smith|  West|Smartphone|Electronics|       1|       600|        600|
|    1003|2023-01-20|       C003|  Alice Brown|  East|      Desk|  Furniture|       3|       150|        450|
|    1004|2023-01-22|       C001|     John Doe| North|     Chair|  Furniture|       5|        80|        400|
|    1005|2023-01-25|       C004|    Bob Green| South|   Monitor|Electronics|       2|       200|        400|
+--------+----------+-----------+-------------+------+----------+-----------+--------+----------+-----------+

+------+-